In [12]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [13]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Omnirob'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Omnirob is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')

Connected
Simulation time in milliseconds:  606356
Omnirob is at [x,y,z]= [7.624989986419678, 8.503308296203613, 0.35449913144111633]
Disconnected


In [14]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [15]:
def move(name, init_velocity):
    # Joint name in coppelia
    motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
    r = robot('Omnirob', motor_names)  # Create an instance of our robot

    # Initial locations
    robot_position = r.get_position()
#     print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

    desired_position = r.get_object_position(name)
    # print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

    # relative_position = r.get_object_relative_position('Sphere1')
    # print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

#     desired_position = [10.00, -7.00, 0]

    # Virtual disk
    disk = robot('Disc', client_id=r.client_id)


    # Time duration
    tf = 10
    # Time interval
    time_steps = linspace(0, tf, 500)

    a0 = robot_position
#     a1 = [1, -1, 0]
    a1 = init_velocity
    a2 = (np.dot(3,desired_position)-3*robot_position-np.dot(a1,2*tf))/tf**2
    a3 = (2*robot_position+np.dot(a1,tf)-np.dot(2,desired_position))/tf**3

    for t in time_steps:
        point_traj = a0 + np.dot(a1,t) + np.dot(a2,t*t) + np.dot(a3,t*t*t)
        vel_traj = a1 + 2*a2*t + 3*a3*t*t
#         print(point_traj)

        disk.set_position(point_traj)


        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        u = 10 * (point_traj - robot_position) + vel_traj

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])



        time.sleep(tf/500)

    r.send_motor_velocities([0, 0, 0, 0])

    r.close_connection() 

In [36]:
def move_to_position(desired_position, init_velocity):
    # Joint name in coppelia
    motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
    r = robot('Omnirob', motor_names)  # Create an instance of our robot

    # Initial locations
    robot_position = r.get_position()
#     print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

    # desired_position = r.get_object_position('Sphere1')
    # print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

    # relative_position = r.get_object_relative_position('Sphere1')
    # print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

#     desired_position = [10.00, -7.00, 0]

    # Virtual disk
    disk = robot('Disc', client_id=r.client_id)


    # Time duration
    tf = 10
    # Time interval
    time_steps = linspace(0, tf, 500)

    a0 = robot_position
#     a1 = [1, -1, 0]
    a1 = init_velocity
    a2 = (np.dot(3,desired_position)-3*robot_position-np.dot(a1,2*tf))/tf**2
    a3 = (2*robot_position+np.dot(a1,tf)-np.dot(2,desired_position))/tf**3

    for t in time_steps:
        point_traj = a0 + np.dot(a1,t) + np.dot(a2,t*t) + np.dot(a3,t*t*t)
        vel_traj = a1 + 2*a2*t + 3*a3*t*t
#         print(point_traj)

        disk.set_position(point_traj)


        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        u = 10 * (point_traj - robot_position) + vel_traj

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])



        time.sleep(tf/500)

    r.send_motor_velocities([0, 0, 0, 0])

    r.close_connection() 

In [87]:
move('Sphere1',(-1,0.5,0))



Robot connected
Robot connected
Connection closed


In [88]:
move('Sphere',(2.5,0,0))

Robot connected
Robot connected
Connection closed


In [89]:
move('Sphere2',(3,0.5,0))

Robot connected
Robot connected
Connection closed


In [90]:
move('Sphere3',(-3.2,-1,0))

Robot connected
Robot connected
Connection closed


In [91]:
move('Sphere0',(-2.3,-1.1,0))

Robot connected
Robot connected
Connection closed


In [92]:
move_to_position((0, 0, 0),(-0.2,-1,0))

Robot connected
Robot connected
Connection closed


In [93]:
move_to_position((-3.5, -5, 0),(0,-1,0))

Robot connected
Robot connected
Connection closed


In [94]:
move('Sphere4',(0,-1,0))

Robot connected
Robot connected
Connection closed
